In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, 
                        summarize, 
                        poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import (cross_validate, 
    KFold, 
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

# 5.3.1 The Validation Set Approach

In [3]:
Auto = load_data("Auto")
Auto_train, Auto_valid = train_test_split(Auto, # specify the dataset
                                          test_size=len(Auto) // 2, 
                                          random_state=0) # the random seed

In [4]:
# fit the model based on the training set
hp_mm = MS(["horsepower"])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train["mpg"]
model = sm.OLS(y_train, X_train) # linear regression
results = model.fit()

In [5]:
# compute the validation MSE
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid["mpg"]
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)

23.61661706966988

In [6]:
# estimate the validation error for higher-degree polynomial regression
def evalMSE(terms, # predictors list
            response, # response name
            train, # training set dataframe
            test # validation set dataframe
           ): 
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred) ** 2)

In [7]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)): 
    MSE[idx] = evalMSE([poly("horsepower", degree)], 
                        "mpg", 
                        Auto_train, 
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
# try different train_test_split
Auto_train, Auto_valid = train_test_split(Auto, # specify the dataset
                                          test_size=len(Auto) // 2, 
                                          random_state=3) # the random seed
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)): 
    MSE[idx] = evalMSE([poly("horsepower", degree)], 
                        "mpg", 
                        Auto_train, 
                        Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

# 5.3.2 Cross-Validation

In [9]:
# the use of wrapper
# we want to use sklearn to perform CV
# but also want to use statsmodels to fit GLMs
hp_model = sklearn_sm(sm.OLS, 
                      MS(["horsepower"]), 
                      # model_args=; the parameter used to pass model arguments, e.g. "family":sm.families.Binomial()
                     )
X, Y = Auto.drop(columns=["mpg"]), Auto["mpg"]
cv_results = cross_validate(hp_model, 
                            X, 
                            Y, 
                            cv=len(Auto)) # specify the interger in K-Fold, here the integer equals n-observations, i.e. LOOCV
cv_err = np.mean(cv_results["test_score"])
cv_err

24.23151351792924

In [10]:
cv_error = np.zeros(5)
H = np.array(Auto["horsepower"])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)): 
    X = np.power.outer(H, np.arange(d + 1))
    # creates a larger matrix of X^0, X^1, X^2, ..., X^d
    # print(X.shape)
    M_CV = cross_validate(M, 
                            X, 
                            Y, 
                            cv=len(Auto))
    cv_error[i] = np.mean(M_CV["test_score"])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In [11]:
# explanation of outer() and np.arange()
A = np.array([3, 5, 9])
np.power.outer(A, np.arange(3 + 1))
# so the first column is used as the "intercept" of the model

array([[  1,   3,   9,  27],
       [  1,   5,  25, 125],
       [  1,   9,  81, 729]])

In [12]:
# K-Fold
cv_error = np.zeros(5)
cv = KFold(n_splits=10, 
           shuffle=True, 
           random_state=0) # use the same split for each polynomial degree
for i, d in enumerate(range(1, 6)): 
    X = np.power.outer(H, np.arange(d + 1))
    M_CV = cross_validate(M, 
                            X, 
                            Y, 
                            cv=cv)
    cv_error[i] = np.mean(M_CV["test_score"])

cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

In [13]:
# use cross_validate to perform validation set approach
validation = ShuffleSplit(n_splits=1, 
                          test_size=len(Auto) // 2, 
                          random_state=0)
results = cross_validate(hp_model, 
                         Auto.drop(["mpg"], axis=1), 
                         Auto["mpg"], 
                         cv=validation)
results["test_score"]

array([23.61661707])

In [14]:
# estimate the variability in the test error
validation = ShuffleSplit(n_splits=10, 
                          test_size=len(Auto) // 2, 
                          random_state=0)
results = cross_validate(hp_model, 
                         Auto.drop(["mpg"], axis=1), 
                         Auto["mpg"], 
                         cv=validation)
results["test_score"].mean(), results["test_score"].std()

(23.802232661034168, 1.421845094109185)

# 5.3.3 The Bootstrap

## Estimating the Accuracy of a Statistic of Interest

In [15]:
Portfolio = load_data("Portfolio")
# calculate alpha using alpha = (σY^2 - σXY) / (σ2X + σ2Y - 2 * σXY)
def alpha_calc(D, idx):
    cov_ = np.cov(D[["X", "Y"]].loc[idx], rowvar=False) # rowvar = False, i.e. each col is a variable
    return ((cov_[1, 1] - cov_[0, 1]) / 
            (cov_[0, 0] + cov_[1, 1] - 2 * cov_[0, 1]))

In [16]:
?np.cov

Signature:
np.cov(
    m,
    y=None,
    rowvar=True,
    bias=False,
    ddof=None,
    fweights=None,
    aweights=None,
    *,
    dtype=None,
)
Docstring:
Estimate a covariance matrix, given data and weights.

Covariance indicates the level to which two variables vary together.
If we examine N-dimensional samples, :math:`X = [x_1, x_2, ... x_N]^T`,
then the covariance matrix element :math:`C_{ij}` is the covariance of
:math:`x_i` and :math:`x_j`. The element :math:`C_{ii}` is the variance
of :math:`x_i`.

See the notes for an outline of the algorithm.

Parameters
----------
m : array_like
    A 1-D or 2-D array containing multiple variables and observations.
    Each row of `m` represents a variable, and each column a single
    observation of all those variables. Also see `rowvar` below.
y : array_like, optional
    An additional set of variables and observations. `y` has the same form
    as that of `m`.
rowvar : bool, optional
    If `rowvar` is True (default), then each row re

In [17]:
alpha_calc(Portfolio, range(100))

0.57583207459283

In [18]:
# randomly select 100 observations from range(100), similar with the bootstrap sample construction
rng = np.random.default_rng(0)
alpha_calc(Portfolio, rng.choice(100, 
                                 100, 
                                 replace=True))

0.6074452469619004

In [19]:
# the function for computing the bootstrap standard error for arbitrary functions
def boot_SE(func, 
            D, 
            n=None, 
            B=1000, 
            seed=0): 
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0] # use the bigger one as n
    for _ in range(B): # repeat sampling for B times
        idx = rng.choice(D.index, 
                         n, 
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value ** 2

    return np.sqrt(second_ / B - (first_ / B) ** 2)

In [20]:
alpha_SE = boot_SE(alpha_calc, 
                   Portfolio, 
                   B=1000, 
                   seed=0)
alpha_SE

0.09118176521277699

## Estimating the Accuracy of a Linear Regression Model

In [21]:
def boot_OLS(model_matrix, response, D, idx): 
    D_ = D.iloc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [22]:
# use partial() function to freeze the arguments which do not change during the bootstrap process
hp_func = partial(boot_OLS, MS(["horsepower"]), "mpg")
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x17b50e560>, ModelSpec(terms=['horsepower']), 'mpg')
File:           /opt/anaconda3/envs/stat_learning/lib/python3.10/functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [23]:
# a demonstration on 10 bootstrap samples
rng = np.random.default_rng(0)
np.array([hp_func(Auto, 
                  rng.choice(len(Auto), 
                             len(Auto), 
                             replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [24]:
# SE of linear coefficients obtained by bootstrap
hp_SE = boot_SE(hp_func, 
                Auto, 
                B=1000, 
                seed=10)
hp_SE

intercept     0.848807
horsepower    0.007352
dtype: float64

In [25]:
# SE of linear coefficients by model summary
hp_model.fit(Auto, Auto["mpg"])
model_SE = summarize(hp_model.results_)["std err"]
model_SE

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [26]:
# difference between SE obtained by bootstrap and model summary for a quadratic model
quad_model = MS([poly("horsepower", 2, raw=True)])
quad_func = partial(boot_OLS, 
                    quad_model, 
                    "mpg")
quad_SE = boot_SE(quad_func, Auto, B=1000)
quad_SE

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [27]:
M = sm.OLS(Auto["mpg"], quad_model.fit_transform(Auto))
summarize(M.fit())["std err"]

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64

In [28]:
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
388,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
389,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
390,28.0,4,120.0,79,2625,18.6,82,1,ford ranger
